In [ ]:
!pip install transformers
!pip install textattack
# !pip install fasttext
!pip install spacy
!pip install pyspellchecker
!pip install textblob

     |████████████████████████████████| 1.9MB 19.1MB/s 
     |████████████████████████████████| 890kB 54.7MB/s 
     |████████████████████████████████| 3.2MB 44.2MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp37-none-any.whl size=893262 sha256=191e98a04bc1c0f49f16c1f096fd4d69832de9690d58f722bbd051f600926254
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses
     |████████████████████████████████| 358kB 18.8MB/s 
     |████████████████████████████████| 184kB 49.0MB/s 
     |████████████████████████████████| 61kB 10.6MB/s 
     |████████████████████████████████| 337kB 51.7MB/s 
     |████████████████████████████████| 102kB 15.0MB/s 
     |████████████████████████████████| 20.1MB 1.3MB/s 
     |████████████████████████████████| 778kB 48.5MB/s 
     |████████████████████████████████| 112kB 51.8MB/s 
     |████████████████████████████████| 245kB 59.2MB/s 
     |█████████████████████████

     |████████████████████████████████| 2.5MB 16.1MB/s 


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import sys
sys.path.append('/content/drive/My Drive/Adversarial')

In [ ]:
from Code.defense.generate_typos import GenerateTypos
from Code.defense.preprocess import Preprocess
from Code.defense.main_defense import MainDefense

In [ ]:
import transformers
from textattack.models.tokenizers import AutoTokenizer
from textattack.models.wrappers import HuggingFaceModelWrapper

model = transformers.AutoModelForSequenceClassification.from_pretrained("textattack/bert-base-uncased-rotten_tomatoes")
tokenizer = AutoTokenizer("textattack/bert-base-uncased-rotten_tomatoes")

model_wrapper = HuggingFaceModelWrapper(model, tokenizer)

# Create the goal function using the model
from textattack.goal_functions import UntargetedClassification
goal_function = UntargetedClassification(model_wrapper)

textattack: Unknown if model of class <class 'transformers.models.bert.modeling_bert.BertForSequenceClassification'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.


In [ ]:
# Import the dataset
from textattack.datasets import HuggingFaceDataset
dataset_train = HuggingFaceDataset("rotten_tomatoes", None, "train")
dataset_val = HuggingFaceDataset("rotten_tomatoes", None, "validation")
dataset_test = HuggingFaceDataset("rotten_tomatoes", None, "test")

Using custom data configuration default
Reusing dataset rotten_tomatoes_movie_review (/root/.cache/huggingface/datasets/rotten_tomatoes_movie_review/default/1.0.0/9198dbc50858df8bdb0d5f18ccaf33125800af96ad8434bc8b829918c987ee8a)
textattack: Loading datasets dataset rotten_tomatoes, split train.
Using custom data configuration default
Reusing dataset rotten_tomatoes_movie_review (/root/.cache/huggingface/datasets/rotten_tomatoes_movie_review/default/1.0.0/9198dbc50858df8bdb0d5f18ccaf33125800af96ad8434bc8b829918c987ee8a)
textattack: Loading datasets dataset rotten_tomatoes, split validation.
Using custom data configuration default
Reusing dataset rotten_tomatoes_movie_review (/root/.cache/huggingface/datasets/rotten_tomatoes_movie_review/default/1.0.0/9198dbc50858df8bdb0d5f18ccaf33125800af96ad8434bc8b829918c987ee8a)
textattack: Loading datasets dataset rotten_tomatoes, split test.


**Define the case constraint**

In [ ]:
import math

from textattack.constraints import Constraint


class CheckCase(Constraint):
    
    def __init__(
        self, compare_against_original=True
    ):
        super().__init__(compare_against_original)
        if not compare_against_original:
            raise ValueError(
                "Cannot apply constraint MaxWordsPerturbed with `compare_against_original=False`"
            )

        # self.isuppercase = False
        # self.islowercase = False

    @staticmethod    
    def all_words_diff(A, B): 
      un_comm = [i for i in "".join(B).split() if i not in "".join(A).split()] 
      return un_comm 

    def _check_constraint(self, transformed_text, reference_text):
        words_diff = list(set(transformed_text.words) - set(reference_text.words))
        # print(transformed_text.words)
        # print(reference_text.words)
        # print(words_diff)
        # print('\n')
        for word in words_diff:
          isuppercase = False
          islowercase = False
          for ch in word[1:]:
            # print(ch)
            if ch.isupper():
              isuppercase = True
            elif ch.islower():
              islowercase = True
            # print(isuppercase)
            # print(islowercase)
            if (isuppercase and islowercase):
              return False
        return True
        # print(not (isuppercase and islowercase))
        # return not (isuppercase and islowercase)

**Define the attack**

In [ ]:
from textattack.search_methods import GreedySearch, BeamSearch
from textattack.shared import Attack
from textattack.constraints.overlap import MaxWordsPerturbed
from textattack.constraints.pre_transformation import (
    MinWordLength,
    RepeatModification,
    StopwordModification,
)
from textattack.goal_functions import UntargetedClassification
from textattack.transformations import (
    CompositeTransformation,
    WordSwapNeighboringCharacterSwap,
    WordSwapQWERTY,
    WordSwapRandomCharacterDeletion,
    WordSwapRandomCharacterInsertion,
)

# from textattack.attack_recipes.pruthi_2019 import Pruthi2019
# from textattack.attack_recipes.deepwordbug_gao_2018 import DeepWordBugGao2018
# from textattack.attack_recipes.textbugger_li_2018 import TextBuggerLi2018

transformation = CompositeTransformation(
            [
                WordSwapNeighboringCharacterSwap(
                    random_one=False, skip_first_char=True, skip_last_char=True
                ),
                # WordSwapRandomCharacterDeletion(
                #     random_one=False, skip_first_char=True, skip_last_char=True
                # ),
                # WordSwapRandomCharacterInsertion(
                #     random_one=False, skip_first_char=True, skip_last_char=True
                # ),
                # WordSwapQWERTY(
                #     random_one=False, skip_first_char=True, skip_last_char=True
                # ),
            ]
        )
# only edit words of length >= 4, edit max_num_word_swaps words.
# note that we also are not editing the same word twice, so
# max_num_word_swaps is really the max number of character
# changes that can be made. The paper looks at 1 and 2 char attacks.
max_num_word_swaps = 5
constraints = [
    MinWordLength(min_length=4),
    StopwordModification(),
    MaxWordsPerturbed(max_num_words=max_num_word_swaps),
    RepeatModification(),
    CheckCase()
]
# untargeted attack
goal_function = UntargetedClassification(model_wrapper)
search_method = GreedySearch()
attack = Attack(goal_function, constraints, transformation, search_method)


# attack = Pruthi2019.build(model_wrapper, max_num_word_swaps=5)
# attack = DeepWordBugGao2018.build(model_wrapper)
# attack = TextBuggerLi2018.build(model_wrapper)


textattack: Unknown if model of class <class 'transformers.models.bert.modeling_bert.BertForSequenceClassification'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.


**Gathering the indices**

In [ ]:
indices_list = []
itr = 0

# for i in range(100000):
try:
  text_input, ground_truth_output = dataset_train[4265]
  # print(text_input)
  print(ground_truth_output)
  # print('\n')
except:
  print("An exception occurred")
  # break
# if len(text_input['text'].split(' ')) < 500:
#   indices_list.append(i)
  itr += 1
# print(len(indices_list))
print(itr)

0
0


In [ ]:
indices_list = list(range(100))
indices_list = indices_list + list(range(4265, 4365))
len(indices_list)

200

**Generating adversarials**

In [ ]:
from tqdm import tqdm # tqdm provides us a nice progress bar.
from textattack.loggers import CSVLogger # tracks a dataframe for us.
from textattack.attack_results import SuccessfulAttackResult
from collections import deque 
from textattack.shared.attacked_text import AttackedText

# indices_list = []
# i = 0
# while len(indices_list) != 1100:
#   text_input, ground_truth_output = dataset[i]
#   if len(text_input['text'].split(' ')) < 500:
#     indices_list.append(i)
#   i += 1
  
# indices_list = indices_list[877:1100]
# indices = deque(indices_list)

results_iterable = attack.attack_dataset(dataset_train, indices=indices_list)

# logger = CSVLogger(color_method='html')

num_successes = 0
adv_examples = []
while num_successes < 200:
    result = next(results_iterable)
    # if isinstance(result, SuccessfulAttackResult):
    # logger.log_attack_result(result)
    adv_examples.append(result)
    num_successes += 1
    print(f'{num_successes} of 200 successes complete.')
        # print('\n')

1 of 200 successes complete.
2 of 200 successes complete.
3 of 200 successes complete.
4 of 200 successes complete.
5 of 200 successes complete.
6 of 200 successes complete.
7 of 200 successes complete.
8 of 200 successes complete.
9 of 200 successes complete.
10 of 200 successes complete.
11 of 200 successes complete.
12 of 200 successes complete.
13 of 200 successes complete.
14 of 200 successes complete.
15 of 200 successes complete.
16 of 200 successes complete.
17 of 200 successes complete.
18 of 200 successes complete.
19 of 200 successes complete.
20 of 200 successes complete.
21 of 200 successes complete.
22 of 200 successes complete.
23 of 200 successes complete.
24 of 200 successes complete.
25 of 200 successes complete.
26 of 200 successes complete.
27 of 200 successes complete.
28 of 200 successes complete.
29 of 200 successes complete.
30 of 200 successes complete.
31 of 200 successes complete.
32 of 200 successes complete.
33 of 200 successes complete.
34 of 200 successes

In [ ]:
len(adv_examples)

200

In [ ]:
adv_labels = list([1] * 100) + list([0] * 100)
# adv_labels

In [ ]:
new_train_text = []
new_train_label = []

for i in range(8530):
  text_input, ground_truth_output = dataset_train[i]
  new_train_text.append(text_input['text'])
  new_train_label.append(ground_truth_output)

# print(adv_examples[0])
new_train_text = new_train_text + [adv.perturbed_text() for adv in adv_examples]
# new_train_text = new_train_text
new_train_label = new_train_label + adv_labels
# new_train_label = new_train_label
# new_train_text
# len(new_train_label)

from transformers import BertTokenizerFast
new_tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')

train_encodings = new_tokenizer(new_train_text, truncation=True, padding=True)

import torch

class CreateDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

new_dataset_train = CreateDataset(train_encodings, new_train_label)

In [ ]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir='/content/drive/My Drive/Adversarial/Dataset/results',          # output directory
    num_train_epochs=2,              # total number of training epochs
    per_device_train_batch_size=32,  # batch size per device during training
    per_device_eval_batch_size=64,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir=None,            # directory for storing logs
    logging_steps=10,
)

trainer = Trainer(
    model=model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=new_dataset_train,         # training dataset
    eval_dataset=dataset_val             # evaluation dataset
)

trainer.train()

Step,Training Loss
10,0.055800
20,0.040800
30,0.044600
40,0.052100
50,0.063100
60,0.060100
70,0.062600
80,0.039800
90,0.024100
100,0.011300


TrainOutput(global_step=546, training_loss=0.05343089422599955, metrics={'train_runtime': 258.735, 'train_samples_per_second': 2.11, 'total_flos': 894622605495840, 'epoch': 2.0})

**Saving adversarials**

In [ ]:
import pickle

with open('/content/drive/My Drive/Adversarial/Dataset/tomato/tomato_bert_swap', 'rb') as handle:
    test_adv_examples = pickle.load(handle)


In [ ]:
from textattack.loggers import CSVLogger # tracks a dataframe for us.
from textattack.attack_results import SuccessfulAttackResult

# logger = CSVLogger(color_method='html')
i = 0
for result in adv_examples[500:1000]:
    if isinstance(result, SuccessfulAttackResult):
      i += 1
print(i)
    # logger.log_attack_result(result)
        # adv_examples.append(result)
        # num_successes += 1
        # print(f'{num_successes} of {len(indices_list)} successes complete.')

242


**Baseline**

In [ ]:
from spellchecker import SpellChecker
from textblob import TextBlob
   
spell = SpellChecker()

def spell_checker_baseline(text):
  splited = text.split(' ')
  corrected = []
  for word in splited:
    b = TextBlob(word)
    # corrected.append(spell.correction(word))
    corrected.append(str(b.correct()))
  return ' '.join(corrected)
  
# find those words that may be misspelled
misspelled = spell.unknown(['something', 'is', 'excelllent', 'here'])
misspelled = ['something', 'is', 'excelllent', 'here']
misspelled = 'something is excelllent here'
spell_checker_baseline(misspelled)

ModuleNotFoundError: ignored

**Run our approach**

In [ ]:
import torch
import time 
device = torch.device("cuda")
true_defenses = 0

defended = []
for i, adv_example in enumerate(test_adv_examples): 

  prt_txt = adv_example.perturbed_text()

  # Pass the prt_text to the 
  t1 = time.time()
  # fixed_txt = Defense.check_input(prt_txt)
  # fixed_txt = spell_checker_baseline(prt_txt)
  # defended.append(fixed_txt)
  fixed_txt = prt_txt
  t2 = time.time()
  print('Time taken for this defense: {} secs'.format(t2 - t1))

  # Check the output of Defense
  # org_txt = adv_example.original_text()
  # fixed_txt = org_txt
  text_input, ground_truth_output = dataset_test[i]
  # org_txt = text_input['text']
  # org_txt = prt_txt
  fixed_adv = tokenizer.encode(fixed_txt)
  # org_input = tokenizer.encode(org_txt)

  fixed_logits = model(input_ids=torch.tensor([fixed_adv['input_ids']]).to(device), attention_mask=torch.tensor([fixed_adv['attention_mask']]).to(device))
  # org_logits = model(input_ids=torch.tensor([org_input['input_ids']]).to(device), attention_mask=torch.tensor([org_input['attention_mask']]).to(device))

  # if torch.sign(fixed_logits.logits[0][0]) == torch.sign(org_logits.logits[0][0]):
  if torch.sign(fixed_logits.logits[0][1] - fixed_logits.logits[0][0]) > 0:
    if ground_truth_output == 1:
      true_defenses += 1
  elif ground_truth_output == 0:
    true_defenses += 1
  else:
    print('******** Unsuccessful Defense **********')
  print(torch.sign(fixed_logits.logits[0][1] - fixed_logits.logits[0][0]))
  print('Total number of adversarials so far: {}'.format(i + 1))
  print('Total number of successful defenses so far: {}'.format(true_defenses))

  with open('/content/drive/My Drive/Adversarial/Dataset/tomato/tomato_bert_insertion_defended', 'wb') as handle:
    pickle.dump(defended, handle, protocol=pickle.HIGHEST_PROTOCOL)
  print('\n')


print('\n')
# print('Acc: {}'.format(true_defenses / len(adv_examples)))

Streaming output truncated to the last 5000 lines.
Total number of adversarials so far: 255
Total number of successful defenses so far: 140


Time taken for this defense: 9.5367431640625e-07 secs
tensor(1., device='cuda:0', grad_fn=<SignBackward>)
Total number of adversarials so far: 256
Total number of successful defenses so far: 141


Time taken for this defense: 7.152557373046875e-07 secs
******** Unsuccessful Defense **********
tensor(-1., device='cuda:0', grad_fn=<SignBackward>)
Total number of adversarials so far: 257
Total number of successful defenses so far: 141


Time taken for this defense: 7.152557373046875e-07 secs
******** Unsuccessful Defense **********
tensor(-1., device='cuda:0', grad_fn=<SignBackward>)
Total number of adversarials so far: 258
Total number of successful defenses so far: 141


Time taken for this defense: 9.5367431640625e-07 secs
tensor(1., device='cuda:0', grad_fn=<SignBackward>)
Total number of adversarials so far: 259
Total number of successful defen

In [ ]:
import pandas as pd
pd.options.display.max_colwidth = 480 # increase colum width so we can actually read the examples

from IPython.core.display import display, HTML
display(HTML(logger.df[['original_text', 'perturbed_text']].to_html(escape=False)))

,original_text,perturbed_text
0,"lovingly photographed in the manner of a golden book sprung to life , stuart little 2 manages sweetness largely without stickiness .","lvoingly photographed in the manner of a golden book sprung to life , stuart little 2 manages sweteness largely wtihout stickiness ."
1,consistently clever and suspenseful .,consistently clveer and suspenesful .
2,"it's like a "" big chill "" reunion of the baader-meinhof gang , only these guys are more harmless pranksters than political activists .","it's like a "" big chill "" reunion of the baader-meinhof gang , only these guys are more harmless pranksters than political activists ."
3,"the story gives ample opportunity for large-scale action and suspense , which director shekhar kapur supplies with tremendous skill .","the story gives ample opportunity for large-scale action and suspense , which director shekhar kapur supplies with treemndous sklil ."
4,"red dragon "" never cuts corners .","red dragon "" never ctus corners ."
5,fresnadillo has something serious to say about the ways in which extravagant chance can distort our perspective and throw us off the path of good sense .,fresnadillo has sometihng seirous to say about the ways in which extravagant chance can distort our perspective and throw us off the path of good sense .
6,throws in enough clever and unexpected twists to make the formula feel fresh .,throws in enough celver and unexpected twists to make the formula feel fersh .
7,weighty and ponderous but every bit as filling as the treat of the title .,weighty and ponderous but every bit as fliling as the treat of the title .
8,"a real audience-pleaser that will strike a chord with anyone who's ever waited in a doctor's office , emergency room , hospital bed or insurance company office .","a real audiencep-leaser that will strike a chrod with anynoe who's eevr waited in a doctor's office , emergency room , hospital bed or insurance company office ."
9,generates an enormous feeling of empathy for its characters .,generates an enomrous feeilng of empathy for its characters .
